# Worldwide Earthquake Events API - Gold Layer Processing

In [2]:
from pyspark.sql.functions import when, col, udf
from pyspark.sql.types import StringType
# ensure the below library is installed on your fabric environment
import reverse_geocoder as rg

StatementMeta(, 7e9f6031-61df-46db-b3d6-a0f8f216cdef, 5, Finished, Available, Finished)

In [3]:
df = spark.read.table("earthquake_events_silver").filter(col('time') > start_date)

StatementMeta(, 7e9f6031-61df-46db-b3d6-a0f8f216cdef, 6, Finished, Available, Finished)

In [4]:
def get_country_code(lat, lon):
    """
    Retrieve the country code for a given latitude and longitude.

    Parameters:
    lat (float or str): Latitude of the location.
    lon (float or str): Longitude of the location.

    Returns:
    str: Country code of the location, retrieved using the reverse geocoding API.

    Example:
    >>> get_country_details(48.8588443, 2.2943506)
    'FR'
    """
    coordinates = (float(lat), float(lon))
    return rg.search(coordinates)[0].get('cc')

StatementMeta(, 7e9f6031-61df-46db-b3d6-a0f8f216cdef, 7, Finished, Available, Finished)

In [5]:
# registering the udfs so they can be used on spark dataframes
get_country_code_udf = udf(get_country_code, StringType())

StatementMeta(, 7e9f6031-61df-46db-b3d6-a0f8f216cdef, 8, Finished, Available, Finished)

In [6]:
# adding country_code
df_with_location = \
                df.\
                    withColumn("country_code", get_country_code_udf(col("latitude"), col("longitude")))

StatementMeta(, 7e9f6031-61df-46db-b3d6-a0f8f216cdef, 9, Finished, Available, Finished)

In [7]:
# adding significance classification
df_with_location_sig_class = \
                            df_with_location.\
                                withColumn('sig_class', 
                                            when(col("sig") < 100, "Low").\
                                            when((col("sig") >= 100) & (col("sig") < 500), "Moderate").\
                                            otherwise("High")
                                            )

StatementMeta(, 7e9f6031-61df-46db-b3d6-a0f8f216cdef, 10, Finished, Available, Finished)

In [12]:
# appending the data to the gold table
df_with_location_sig_class.write.mode('append').saveAsTable('earthquake_events_gold')

StatementMeta(, 7e9f6031-61df-46db-b3d6-a0f8f216cdef, 15, Finished, Available, Finished)